In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
raw_data = pd.read_csv('PopulationData.csv', index_col=0)#.drop(0, axis=0)
raw_data.shape

(3273, 132)

In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3273 entries, 0 to 72153
Columns: 132 entries, State to R_NET_MIG_2017
dtypes: float64(49), object(83)
memory usage: 3.3+ MB


In [4]:
raw_data.head()

,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,...,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017
FIPS,,,,,,,,,,,,,,,,,,,,,
0,US,United States,NaN,NaN,NaN,NaN,NaN,"308,745,538","308,758,105","309,338,421",...,0.0,0.0,0.0,2.7,2.9,2.9,3.2,3.5,3.5,3.4
1000,AL,Alabama,NaN,NaN,NaN,NaN,NaN,"4,779,736","4,780,135","4,785,579",...,-0.3,-0.4,0.8,0.5,0.9,1.4,0.7,0.7,0.6,1.7
1001,AL,Autauga County,2.0,2.0,2.0,2.0,0.0,"54,571","54,571","54,750",...,-2.0,4.8,1.0,5.9,-6.1,-4.1,2.1,-1.7,5.1,1.3
1003,AL,Baldwin County,4.0,3.0,5.0,2.0,5.0,"182,265","182,265","183,110",...,17.0,20.5,22.4,16.3,17.2,22.6,20.4,17.9,21.3,23.2
1005,AL,Barbour County,6.0,6.0,6.0,6.0,3.0,"27,457","27,457","27,332",...,-16.2,-18.8,-19.0,0.3,-6.8,-8.0,-5.5,-16.4,-18.9,-19.2


In [5]:
# Drop unwanted rows and rows with null values
new_data = raw_data.set_index('State').drop('PR', axis=0).drop('US', axis=0).dropna(how='all').reset_index()
print new_data.shape
new_data.head()

(3193, 132)


,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,...,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017
0,AL,Alabama,NaN,NaN,NaN,NaN,NaN,"4,779,736","4,780,135","4,785,579",...,-0.3,-0.4,0.8,0.5,0.9,1.4,0.7,0.7,0.6,1.7
1,AL,Autauga County,2.0,2.0,2.0,2.0,0.0,"54,571","54,571","54,750",...,-2.0,4.8,1.0,5.9,-6.1,-4.1,2.1,-1.7,5.1,1.3
2,AL,Baldwin County,4.0,3.0,5.0,2.0,5.0,"182,265","182,265","183,110",...,17.0,20.5,22.4,16.3,17.2,22.6,20.4,17.9,21.3,23.2
3,AL,Barbour County,6.0,6.0,6.0,6.0,3.0,"27,457","27,457","27,332",...,-16.2,-18.8,-19.0,0.3,-6.8,-8.0,-5.5,-16.4,-18.9,-19.2
4,AL,Bibb County,1.0,1.0,1.0,1.0,0.0,"22,915","22,919","22,872",...,0.9,-1.4,-0.9,-5.0,-4.1,-5.9,1.2,1.8,-0.5,0.0


In [11]:
# Create list to subset the data
c1 = ['State', 'Area_Name']
c2 = ['State', 'Area_Name']
c3 = ['State', 'Area_Name']
for c in new_data.columns:
    if 'POP_ESTIMATE' in c:
        c1.append(c)
    elif 'R_INTERNATIONAL_MIG' in c:
        c2.append(c)
    elif 'R_DOMESTIC_MIG' in c:
        c3.append(c)

In [12]:
pop_data = new_data[c1]
print pop_data.shape
pop_data.head()

(3193, 10)


,State,Area_Name,POP_ESTIMATE_2010,POP_ESTIMATE_2011,POP_ESTIMATE_2012,POP_ESTIMATE_2013,POP_ESTIMATE_2014,POP_ESTIMATE_2015,POP_ESTIMATE_2016,POP_ESTIMATE_2017
0,AL,Alabama,"4,785,579","4,798,649","4,813,946","4,827,660","4,840,037","4,850,858","4,860,545","4,874,747"
1,AL,Autauga County,"54,750","55,199","54,927","54,695","54,864","54,838","55,278","55,504"
2,AL,Baldwin County,"183,110","186,534","190,048","194,736","199,064","202,863","207,509","212,628"
3,AL,Barbour County,"27,332","27,351","27,175","26,947","26,749","26,264","25,774","25,270"
4,AL,Bibb County,"22,872","22,745","22,658","22,503","22,533","22,561","22,633","22,668"


In [13]:
pop_data = pop_data.set_index(['State', 'Area_Name']).stack().reset_index()
pop_data.columns = ['State', 'Area', 'Year', 'Pop_Est']
pop_data.head(15)

,State,Area,Year,Pop_Est
0,AL,Alabama,POP_ESTIMATE_2010,"4,785,579"
1,AL,Alabama,POP_ESTIMATE_2011,"4,798,649"
2,AL,Alabama,POP_ESTIMATE_2012,"4,813,946"
3,AL,Alabama,POP_ESTIMATE_2013,"4,827,660"
4,AL,Alabama,POP_ESTIMATE_2014,"4,840,037"
5,AL,Alabama,POP_ESTIMATE_2015,"4,850,858"
6,AL,Alabama,POP_ESTIMATE_2016,"4,860,545"
7,AL,Alabama,POP_ESTIMATE_2017,"4,874,747"
8,AL,Autauga County,POP_ESTIMATE_2010,"54,750"
9,AL,Autauga County,POP_ESTIMATE_2011,"55,199"


In [15]:
# Function to remove nondigits from values in a column
def remove_nondigits(col):
    values = []
    for val in col:
        val = str(val)
        newval = re.sub(r'\D', "", val)
        values.append(int(newval))
    return pd.Series(values)

In [16]:
pop_data.Year = remove_nondigits(pop_data.Year)

In [17]:
pop_data.head()

,State,Area,Year,Pop_Est
0,AL,Alabama,2010,"4,785,579"
1,AL,Alabama,2011,"4,798,649"
2,AL,Alabama,2012,"4,813,946"
3,AL,Alabama,2013,"4,827,660"
4,AL,Alabama,2014,"4,840,037"


In [18]:
i_mig_data = new_data[c2]
i_mig_data = i_mig_data.set_index(['State','Area_Name']).stack().reset_index()
i_mig_data.columns = ['State', 'Area', 'Year', 'Int_Mig_Rate']

In [19]:
i_mig_data.Year = remove_nondigits(i_mig_data.Year)
i_mig_data.head()

,State,Area,Year,Int_Mig_Rate
0,AL,Alabama,2011,0.9
1,AL,Alabama,2012,0.9
2,AL,Alabama,2013,0.8
3,AL,Alabama,2014,0.8
4,AL,Alabama,2015,1.0


In [20]:
d_mig_data = new_data[c3]
d_mig_data = d_mig_data.set_index(['State', 'Area_Name']).stack().reset_index()
d_mig_data.columns = ['State', 'Area', 'Year', 'Dom_Mig_Rate']

In [21]:
d_mig_data.Year = remove_nondigits(d_mig_data.Year)
d_mig_data.head()

,State,Area,Year,Dom_Mig_Rate
0,AL,Alabama,2011,-0.5
1,AL,Alabama,2012,0.0
2,AL,Alabama,2013,0.5
3,AL,Alabama,2014,-0.2
4,AL,Alabama,2015,-0.3


In [22]:
new_df = pd.merge(pop_data, i_mig_data, on=['State', 'Area', 'Year'])

In [23]:
new_df.head()

,State,Area,Year,Pop_Est,Int_Mig_Rate
0,AL,Alabama,2011,"4,798,649",0.9
1,AL,Alabama,2012,"4,813,946",0.9
2,AL,Alabama,2013,"4,827,660",0.8
3,AL,Alabama,2014,"4,840,037",0.8
4,AL,Alabama,2015,"4,850,858",1.0


In [24]:
new_df.Pop_Est = remove_nondigits(new_df.Pop_Est)

In [25]:
clean_df = pd.merge(new_df, d_mig_data, on = ['State', 'Area', 'Year'])

In [26]:
print clean_df.shape
print clean_df.dtypes
clean_df.head()

(22393, 6)
State            object
Area             object
Year              int64
Pop_Est           int64
Int_Mig_Rate    float64
Dom_Mig_Rate    float64
dtype: object


,State,Area,Year,Pop_Est,Int_Mig_Rate,Dom_Mig_Rate
0,AL,Alabama,2011,4798649,0.9,-0.5
1,AL,Alabama,2012,4813946,0.9,0.0
2,AL,Alabama,2013,4827660,0.8,0.5
3,AL,Alabama,2014,4840037,0.8,-0.2
4,AL,Alabama,2015,4850858,1.0,-0.3


In [27]:
clean_df.to_csv('Pop_Mig_Data.csv')